In [1]:
from excellib import set_excel_color_daily
from stocklib import get_stock_price, get_OTC_price, get_otc, get_investor, get_ratio
from stocklib import cal_bar, cal_KD, check_10days, check_10days_otc, get_trust_today
from stocklib import convert_num
from stock_plot import plot_K_chart


from finance import get_yearly, get_df_monthly, get_df_monthly_check, get_df_quarterly
from finance import get_this_quarter, get_quarter_income, get_quarter_finance, set_growth_eps
from finance import get_df_quarterly_all, get_df_yearly_all, get_estimate_income
from finance import check_5_years, get_df_quarterly_equity, get_finance_col, get_df_yearly

In [2]:
import requests
import shutil
import time
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import yfinance as yf
import json
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
from tqdm import tqdm
import os
%matplotlib inline

ua = UserAgent()
root = '../'

root = 'D:/Mia/Stock/'
path_quarterly = root + 'quarterly/'
path_monthly = root + 'monthly/'
path_month_finance = root + 'monthly_finance/'
path_yearly = root + 'yearly/'
path_finance = root + 'finance/'
path_trust = root + 'trust/'
folder_name = 'trust/'

In [20]:
d_today = date.today() #- timedelta(days=1)

today_tw = d_today - relativedelta(years=1911)
this_year = d_today.year
this_quarter = get_this_quarter()
n = 1
if d_today.weekday() == 0:
    n = 3
d_yesterday = d_today - timedelta(days=n)
s_yesterday = (d_today - timedelta(days=n)).strftime("%Y%m%d")
s_today_md = (d_today).strftime("%m/%d")
s_yesterday_md = (d_today - timedelta(days=n)).strftime("%m/%d")
s_today = d_today.strftime("%Y%m%d")
s_today_hm = d_today.strftime("%Y%m%d_%H%M")

s_today_tw = today_tw.strftime("%Y/%m/%d").lstrip('0')
s_thismonth_tw = today_tw.strftime("%Y/%m").lstrip('0')
print('今天:', d_today, s_today_hm)
# 台灣證券交易所

今天: 2022-03-30 20220330_0000


In [14]:
df_today = get_stock_price(d_today)
df_today.head(2)

取得今日上市股票資訊: 20220330


,name,amount,open,high,low,close
id,,,,,,
0050,元大台灣50,8498,138.05,138.7,138.05,138.55
0051,元大中型100,41,60.00,60.3,60.00,60.30


In [7]:
def get_EPS(lst_stock_id, filename):
    df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍
    df_quarterly_all = get_df_quarterly_all()
    df_estimate = pd.DataFrame(columns = list(df_quarterly_all.columns)+['普通股數'])

    # 取得公司發行股數
    path_company_stock_num = root + 'company_list.xlsx'
    df_company_stock_num = pd.read_excel(path_company_stock_num, engine='openpyxl', dtype={'stock_id':str})
    df_company_stock_num.set_index('stock_id', inplace=True)
    
    lst_eps = []
    for i in tqdm(range(len(lst_stock_id))):
        stock_id = lst_stock_id[i]
        t_wait = np.random.randint(5,15)
        print(stock_id, ', wait ', t_wait)
        df = get_df_quarterly(stock_id)

        if df is None:
            print('df is None')
            continue
        if df.loc[df.index[0], '毛利率(%)'] < 20:
            print(stock_id, '：毛利率', df.loc[df.index[0], '毛利率(%)'], '< 20%，跳過')
            continue
        elif df.loc[df.index[0], '營益率(%)'] < 8:
            print(stock_id, '：營益率' , df.loc[df.index[0], '營益率(%)'] , ' < 8%，跳過')
            continue

        lst_eps.append(stock_id)
        # 計算歷年同季的營收成長率、EPS
        df = set_growth_eps(df, df_today, stock_id)

        # 將年度EPS寫到歷年excel裡
        lst_year = set(df.index.to_series().apply(lambda x:x[:4]))
        min_year = int(min(lst_year)) # 若最小年是2018 ，只能確定2019都有資料
        lst_year = [y for y in lst_year if int(y) > min_year]
        lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]

        df.reset_index(inplace=True)
        df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

        stock_name = ""
        if stock_id in df_today.index:
            stock_name = df_today.loc[stock_id, 'name']
            df_yearly = get_yearly(stock_id, lst_EPS, df['quarter'][0][:4], stock_name)    
            df_yearly_all = df_yearly_all.append(df_yearly, ignore_index=True)

        df.insert(0, 'name', value=stock_name)
        df.insert(0, 'stock_id', value=stock_id)

        df_quarterly_all = df_quarterly_all.append(df, ignore_index=True)
        time.sleep(t_wait)

        # ===============================
        # 預測EPS ( check_EPS )
        # ===============================
        df_new_quarter = get_quarter_finance(stock_id, df, this_year, this_quarter, df_company_stock_num, df_today)
        df_concat = pd.concat([df, df_new_quarter])
        df_concat.sort_values('quarter', ascending=False, inplace=True)
        df_estimate = df_estimate.append(df_concat)

    #預測EPS ( check_EPS )
    s_now = datetime.now().strftime("%Y%m%d_%H%M")
    path = path_check+'estimate_{}_{}.xlsx'.format(filename, s_now)
    print(path)
    df_estimate.to_excel(path, index=False)
    return lst_eps

In [8]:
path_check = path_quarterly

# # 月報
# #### 彙總報表 (單位：千元)
# https://mops.twse.com.tw/mops/web/t21sc04_ifrs

# # 從月報 篩選出公司
# 上個月營收 > 30億
# 上個月營收成長 > 20% or 去年同季營收成長 > 20%
df_monthly_check = get_df_monthly_check(path_month_finance, income=5, income_growth_MoM=10, income_growth_YoY=10, pre_month=False)
df_monthly_check.head(2)

從月報 篩選出幾家公司
上個月營收 > 5億 上個月營收成長 > 10% or 去年同季營收成長 > 10%
(13429, 14)
目前月報年份： 111/2
所有公司筆數： 971
本月觀察公司筆數： 264


D:\Mia\Stock\code\finance.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_monthly_check['公司代號'] = df_monthly_check['公司代號'].astype(str)


,出表日期,資料年月,公司代號,公司名稱,產業別,營業收入-當月營收,營業收入-上月營收,營業收入-去年當月營收,營業收入-上月比較增減(%),營業收入-去年同月增減(%),累計營業收入-當月累計營收,累計營業收入-去年累計營收,累計營業收入-前期比較增減(%),備註
12464,111/03/30,111/2,1101,台泥,水泥工業,54.56,75.18,41.73,-27.43,30.73,129.74,122.58,5.84,"以上營收不包含停業部門營收:111年2月 0 仟元暨110年2月363,378仟元；111年..."
11499,111/03/30,111/1,1101,台泥,水泥工業,75.18,108.21,80.84,-30.53,-7.01,75.18,80.84,-7.01,"以上營收不包含停業部門營收:111年1月 0 仟元暨110年1月511,796仟元；111年..."


In [11]:
lst_monthly_id = list(df_monthly_check['公司代號'].unique())
if '6770' in lst_monthly_id:
    lst_monthly_id.remove('6770')
print(len(lst_monthly_id))
print(lst_monthly_id)

263
['1101', '1102', '1210', '1215', '1218', '1301', '1303', '1304', '1308', '1309', '1310', '1313', '1314', '1321', '1326', '1338', '1402', '1409', '1434', '1440', '1444', '1447', '1451', '1460', '1464', '1473', '1476', '1477', '1503', '1504', '1507', '1513', '1519', '1522', '1532', '1560', '1582', '1590', '1605', '1608', '1609', '1708', '1709', '1710', '1714', '1717', '1722', '1723', '1725', '1726', '1773', '1802', '1904', '2002', '2006', '2010', '2012', '2013', '2015', '2023', '2027', '2028', '2030', '2031', '2034', '2038', '2049', '2062', '2069', '2106', '2206', '2301', '2303', '2308', '2312', '2313', '2317', '2324', '2327', '2328', '2330', '2331', '2337', '2338', '2344', '2347', '2351', '2352', '2353', '2354', '2356', '2357', '2362', '2368', '2371', '2376', '2377', '2379', '2382', '2383', '2385', '2387', '2388', '2395', '2397', '2401', '2404', '2408', '2419', '2421', '2428', '2454', '2464', '2472', '2481', '2511', '2514', '2535', '2545', '2546', '2548', '2597', '2603', '2606', '26

In [10]:
sep_num = 90
n = 0
lst_all_id = []
if len(lst_monthly_id)>sep_num:
    while n < len(lst_monthly_id):
        lst_all_id.append(lst_monthly_id[n:n+sep_num])
        n += sep_num
else:
    lst_all_id.append(lst_monthly_id)
for i in lst_all_id:
    print(len(i))

90
90
83


# 取得公司歷年發放股利

In [12]:
# import requests
# from bs4 import BeautifulSoup
# from fake_useragent import UserAgent
# from datetime import date 
# from tqdm import tqdm

# path_dividen = root + 'company_dividen.xlsx'
# # df_dividen_all = pd.DataFrame(columns=['stock_id', 'year', 'cash', 'stock'])
# df_dividen_all = pd.read_excel(path_dividen, engine='openpyxl', dtype={'stock_id':str})

# ua = UserAgent()
# agent = ua.random

# for i in range(len(lst_all_id)):
#     print('batch', i)
#     lst_1 = lst_all_id[i]
#     for stock_id in tqdm(lst_1):
#         if stock_id in df_dividen_all['stock_id'].unique(): #or not stock_id in lst_company:
#             continue
#     #     stock_id = lst_company_1[i]
#         t_wait = np.random.randint(15, 25)
#         print(stock_id, 'wait:', t_wait)
#         url = 'https://goodinfo.tw/StockInfo/StockDetail.asp?STOCK_ID={}'.format(stock_id)

#         headers = {'user-agent':agent}
#         res = requests.get(url, headers=headers)
#         res.encoding = 'utf-8'
#         soup = BeautifulSoup(res.text, 'lxml')

#         tb = soup.find(id='FINANCE_DIVIDEND')

#         try:
#             df_dividen = pd.read_html(str(tb))[0]
#         except:
#             print(stock_id, 'Error No tables found')
#             continue
#         df_dividen = df_dividen.iloc[:,:-1]
#         df_dividen.columns = ['year', 'cash', 'stock']
#         df_dividen.drop(df_dividen.index[:3], inplace=True)
#         df_dividen.drop(df_dividen.index[-1], inplace=True)
#         df_dividen['year'] = df_dividen['year'].astype(int)
#         df_dividen = df_dividen[df_dividen['year']>2014]
#         df_dividen.insert(0, 'stock_id', value=stock_id)
#         df_dividen_all = df_dividen_all.append(df_dividen)
#         time.sleep(t_wait)

#     df_dividen_all.to_excel(path_dividen, index=False)

#     # 合併發行股數 與 歷年股利Excel
#     path_company = root + 'company_list.xlsx'
#     path_dividen = root + 'company_dividen.xlsx'
#     path_company_info = root + 'company_info.xlsx'
#     import shutil
#     shutil.copy2(path_company_info, path_company_info.replace('.xlsx', '_backup.xlsx'))

#     df_1 = pd.read_excel(path_company, engine='openpyxl')
#     df_2 = pd.read_excel(path_dividen, engine='openpyxl')
#     print(df_1.shape)
#     print(df_2.shape)
#     df_all = df_1.set_index('stock_id').join(df_2.set_index('stock_id'))
#     print(df_all.shape)
#     df_all.head(2)
#     df_all.reset_index(inplace=True)
#     df_all.to_excel(path_company_info, index=False)

  0%|                                                                                           | 0/90 [00:00<?, ?it/s]

batch 0
2106 wait: 24


100%|█████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 178.94it/s]

2106 Error No tables found


(934, 3)
(3951, 4)
(4262, 5)


...

 27%|█████████████████████▋                                                            | 22/83 [00:00<00:01, 47.27it/s]

5306 Error No tables found
9924 wait: 19


100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [00:00<00:00, 83.40it/s]

9924 Error No tables found


(934, 3)
(3951, 4)
(4262, 5)


In [13]:
# ==============================
#  取得觀察中公司的季報、預測EPS
# ==============================
df_quarterly_all = get_df_quarterly_all()
df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍

lst_eps_pass = []
print('觀察中公司有：', len(lst_monthly_id), '筆')
for i in range(len(lst_all_id)):
#     if i < 2:
#         print('skip:', i)
#         continue
    print('Check:', i)
    lst = lst_all_id[i]
    lst_monthly_id = get_EPS(lst, 'quarterly')
    lst_eps_pass += lst_monthly_id
    print('EPS檢查', len(lst_monthly_id), '筆\n', lst_monthly_id)

  0%|                                                                                           | 0/90 [00:00<?, ?it/s]

觀察中公司有： 263 筆
Check: 0
1101 , wait  8


c:\users\mia\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1101  wait  7


  1%|▉                                                                                  | 1/90 [00:17<25:28, 17.18s/it]

...

9958 , wait  8


c:\users\mia\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


9958  wait  6


100%|██████████████████████████████████████████████████████████████████████████████████| 83/83 [12:39<00:00,  9.15s/it]


EPS檢查 34 筆
 ['4536', '4919', '4927', '4942', '4958', '4961', '5007', '5306', '5522', '5871', '6005', '6202', '6239', '6257', '6271', '6409', '6415', '6579', '6592', '6670', '6691', '6706', '6719', '6781', '8016', '8046', '8070', '8081', '9910', '9911', '9938', '9939', '9945', '9958']


In [4]:
lst_eps_pass =  ['1101', '1102', '1301', '1303', '1304', '1308', '1409', '1476', '1477', '1504', '1513', '1532', '1560', '1590', '1708', '1709', '1773', '1802', '2002', '2027', '2030', '2049', '2303', '2308', '2327', '2330', '2337', '2338', '2344'] \
+ ['2357', '2368', '2376', '2379', '2383', '2395', '2401', '2408', '2428', '2454', '2472', '2481', '2545', '2548', '2603', '2606', '2609', '2610', '2615', '2618', '2636', '2637', '2801', '2809', '2834', '2838', '2886', '2887', '2891', '2897', '2905', '2923', '3005', '3030', '3034', '3035', '3037', '3189', '3406', '3413', '3443', '3532', '3592', '3653', '3665'] \
+ ['4536', '4919', '4927', '4942', '4958', '4961', '5007', '5306', '5522', '5871', '6005', '6202', '6239', '6257', '6271', '6409', '6415', '6579', '6592', '6670', '6691', '6706', '6719', '6781', '8016', '8046', '8070', '8081', '9910', '9911', '9938', '9939', '9945', '9958']
print(lst_eps_pass)

['1101', '1102', '1301', '1303', '1304', '1308', '1409', '1476', '1477', '1504', '1513', '1532', '1560', '1590', '1708', '1709', '1773', '1802', '2002', '2027', '2030', '2049', '2303', '2308', '2327', '2330', '2337', '2338', '2344', '2357', '2368', '2376', '2379', '2383', '2395', '2401', '2408', '2428', '2454', '2472', '2481', '2545', '2548', '2603', '2606', '2609', '2610', '2615', '2618', '2636', '2637', '2801', '2809', '2834', '2838', '2886', '2887', '2891', '2897', '2905', '2923', '3005', '3030', '3034', '3035', '3037', '3189', '3406', '3413', '3443', '3532', '3592', '3653', '3665', '4536', '4919', '4927', '4942', '4958', '4961', '5007', '5306', '5522', '5871', '6005', '6202', '6239', '6257', '6271', '6409', '6415', '6579', '6592', '6670', '6691', '6706', '6719', '6781', '8016', '8046', '8070', '8081', '9910', '9911', '9938', '9939', '9945', '9958']


In [14]:
def check_income_pass(lst_stock_expect, df_quarter, df_month):
    df_expect = get_df_quarterly_all()
    for stock_id in lst_stock_expect:
    # stock_id = '4961'
        df_stock = df_quarter[df_quarter['stock_id']==stock_id]
        df_estimate_income = get_estimate_income(df_stock, stock_id, this_year, this_quarter)

        if stock_id in list(df_month['公司代號']): 
            real_income = round(df_month[df_month['公司代號']==stock_id].iloc[0,5] / 100000, 2)
            expect_income = round(df_estimate_income.iloc[0, 3], 2)

            stock_name = df_estimate_income.iloc[0,1]
            real_q = df_estimate_income.iloc[0,2] +'_r'
            df_real = pd.DataFrame([{'stock_id':stock_id, 'name':stock_name, 'quarter':real_q, '營業收入':real_income}]
                                   ,columns=df_expect_all.columns)
            df_estimate_income = df_estimate_income.append(df_real)
            df_estimate_income.sort_values('quarter', ascending=False, inplace=True)

            if  real_income > expect_income:
                lst_pass.append(stock_id)
            else:
                lst_bad.append(stock_id)
        else:
            lst_unknown.append(stock_id)
        df_expect = df_expect.append(df_estimate_income.iloc[:4,:-5])
    return df_expect

In [8]:
from glob import glob
path_month_finance = path_month_finance + 't21sc03_111_2.csv'
df_month = pd.read_csv(path_month_finance, dtype={'公司代號':str})
lst_pass = []
lst_unknown = []
lst_bad = []
df_expect_all = get_df_quarterly_all()
df_income_all = get_df_quarterly_all()

# for path_expect in glob(path_quarterly+'2021Q4/*'):
for path_expect in glob(path_quarterly+'2022Q2/*'):
    df_quarter = pd.read_excel(path_expect, engine='openpyxl', dtype={'stock_id':str})
    df_expect_all = df_expect_all.append(df_quarter)
    
    df_quarter = df_quarter[ (df_quarter['quarter']==str(this_year)+'Q'+str(this_quarter)) 
                           & (df_quarter['本益比(Y)']<12)]
    lst_stock_expect = list(df_quarter['stock_id'])
    print(lst_stock_expect)
    df_income_all = df_income_all.append(check_income_pass(
                                            lst_stock_expect, df_quarter, df_month))
    

['4935', '4942', '4961', '5538', '5608', '6116', '6177', '6192', '6605', '6670', '6672', '6781', '8016', '8404', '9938', '9940']
['1301', '1303', '1304', '1308', '1532', '1583', '2002', '2027', '2028', '2030', '2031', '2104', '2337', '2344', '2348', '2375', '2376']
['2408', '2441', '2449', '2451', '2467', '2520', '2539', '2542', '2545', '2548', '2601', '2603', '2606', '2609', '2615', '2636', '2637', '3006', '3014', '3034', '3545', '3665']


In [9]:
df_income_pass = df_income_all[df_income_all['stock_id'].isin(lst_pass)]
df_income_pass.head(2)

,stock_id,name,quarter,營業收入,營業成本,營業毛利,營業毛利(毛損)淨額,營業費用,營業利益,稅前淨利,...,股利_avg,股利(%),殖利率(%),2020,2019,2018,2017,2016,2015,2019
0,4935,茂林-KY,2021Q412_r,9.27,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4935,茂林-KY,2021Q412,8.83,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_expect_pass = df_expect_all[df_expect_all['stock_id'].isin(lst_pass)]

df_quarterly_pass = df_expect_pass.append(df_income_pass)
df_quarterly_pass = df_quarterly_pass.sort_values(['stock_id', 'quarter'], ascending=[True, False])
df_quarterly_pass.to_excel(path_quarterly+'2021Q4/quarterly_pass_{}.xlsx'.format(s_today), index=False)
df_quarterly_pass.head(3)

,stock_id,name,quarter,營業收入,營業成本,營業毛利,營業毛利(毛損)淨額,營業費用,營業利益,稅前淨利,...,殖利率(%),2020,2019,2018,2017,2016,2015,普通股數,year,2019
0,1301,台塑,2021Q412_r,251.99,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1301,台塑,2021Q412,246.04,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1301,台塑,2021Q411,245.24,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 5年成長

In [5]:
df_income_pass = pd.read_excel(path_quarterly+'2021Q4/quarterly_pass_20220115.xlsx'
                               , engine='openpyxl', dtype={'stock_id':str})

lst_pass = list(df_income_pass['stock_id'].unique())
# lst_pass = ['3023'] #,'2603' '2609', '2637', '3665'
print(len(lst_pass))
print(lst_pass)

22
['1301', '1532', '1583', '2348', '2375', '2449', '2467', '2520', '2545', '2603', '2609', '2637', '3665', '4935', '4942', '5538', '6116', '6177', '6670', '6781', '9938', '9940']


In [5]:
lst_0 = ['1102', '1227', '1301', '1303', '1304', '1308', '1409', '1476', '1477', '1532', '1536', '1560', '1583', '1590', '1707', '1708', '1718', '1723', '1726', '1773', '1795', '1808', '2002', '2027', '2028', '2030', '2031', '9941', '9958']
lst_1 = ['2049', '2059', '2104', '2207', '2303', '2327', '2330', '2337', '2338', '2344', '2348', '2368', '2369', '2373', '2375', '2376', '2379', '2383', '2395', '2408', '2409', '2428', '2441', '2449', '2451', '2454', '2464', '2467', '2472', '2476', '2478', '2481', '2492', '2511', '2520', '2524', '2539', '2542', '2545', '2548', '2601', '2603', '2606']
lst_2 = ['2609', '2615', '2636', '2637', '2801', '2809', '2812', '2838', '2855', '2880', '2882', '2883', '2884', '2885', '2886', '2887', '2888', '2889', '2890', '2891', '2892', '3005', '3006', '3014', '3019', '3026', '3034', '3035', '3037', '3045', '3189', '3413', '3443', '3515', '3533', '3545', '3592', '3653', '3661', '3665', '3711', '4536', '4755', '4927']
lst_3 = ['4935', '4942', '4958', '4961', '5007', '5519', '5538', '5608', '5871', '6005', '6116', '6177', '6192', '6202', '6206', '6239', '6257', '6271', '6277', '6409', '6415', '6491', '6531', '6579', '6592', '6605', '6670', '6672', '6691', '6706', '6781', '6806', '8016', '8046', '8070', '8081', '8114', '8210', '8404', '9911', '9921', '9938', '9939', '9940']

lst_pass = lst_0 + lst_1 + lst_2 + lst_3
# lst_pass = ['3023']
len(lst_pass)

160

In [5]:
lst_pass = lst_eps_pass

sep_num = 55
n = 0
lst_all_id = []
if len(lst_pass)>sep_num:
    while n < len(lst_pass):
        lst_all_id.append(lst_pass[n:n+sep_num])
        n += sep_num
else:
    lst_all_id.append(lst_pass)
for i in lst_all_id:
    print(len(i))

55
53


In [12]:
# # 計算股東權益
for lst_id in lst_all_id:
    for stock_id in tqdm(lst_id):
        file = root + 'equity/quarterly_equity_{}.xlsx'.format(stock_id)
        if stock_id == '2885':
            continue
#         if os.path.exists(file):
#             print(stock_id, '已存在')
#             continue
        df_equity = get_df_quarterly_equity(stock_id)

  0%|                                                                                           | 0/55 [00:00<?, ?it/s]

1101 add new equity 2021Q4
get_equity： 1101 2021 4
1101 add new equity 2021Q3
get_equity： 1101 2021 3
1101 add new equity 2021Q2
get_equity： 1101 2021 2
1101 add new equity 2021Q1
get_equity： 1101 2021 1
1101 add new equity 2020Q4
get_equity： 1101 2020 4
1101 add new equity 2020Q3
get_equity： 1101 2020 3
1101 add new equity 2020Q2
get_equity： 1101 2020 2
1101 add new equity 2020Q1
get_equity： 1101 2020 1
1101 add new equity 2019Q4
get_equity： 1101 2019 4
1101 add new equity 2019Q3
get_equity： 1101 2019 3
1101 add new equity 2019Q2
get_equity： 1101 2019 2
1101 add new equity 2019Q1
get_equity： 1101 2019 1
1101 add new equity 2018Q4
get_equity： 1101 2018 4
1101 add new equity 2018Q3
get_equity： 1101 2018 3
1101 add new equity 2018Q2
get_equity： 1101 2018 2
1101 add new equity 2018Q1
get_equity： 1101 2018 1


  2%|█▌                                                                                 | 1/55 [00:35<31:32, 35.04s/it]

1102 已存在


c:\users\mia\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
  4%|███                                                                                | 2/55 [00:51<21:07, 23.91s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▉   | 51/53 [01:29<00:02,  1.32s/it]

9945 add new equity 2021Q4
get_equity： 9945 2021 4
9945 查無資料
9945 add new equity 2021Q3
get_equity： 9945 2021 3
9945 查無資料
9945 add new equity 2021Q2
get_equity： 9945 2021 2
9945 查無資料
9945 add new equity 2021Q1
get_equity： 9945 2021 1
9945 查無資料
9945 add new equity 2020Q4
get_equity： 9945 2020 4
9945 查無資料
9945 add new equity 2020Q3
get_equity： 9945 2020 3
9945 查無資料
9945 add new equity 2020Q2
get_equity： 9945 2020 2
9945 查無資料
9945 add new equity 2020Q1
get_equity： 9945 2020 1
9945 查無資料
9945 add new equity 2019Q4
get_equity： 9945 2019 4
9945 查無資料
9945 add new equity 2019Q3
get_equity： 9945 2019 3
9945 查無資料
9945 add new equity 2019Q2
get_equity： 9945 2019 2
9945 查無資料
9945 add new equity 2019Q1
get_equity： 9945 2019 1
9945 查無資料
9945 add new equity 2018Q4
get_equity： 9945 2018 4
9945 查無資料
9945 add new equity 2018Q3
get_equity： 9945 2018 3
9945 查無資料
9945 add new equity 2018Q2
get_equity： 9945 2018 2
9945 查無資料
9945 add new equity 2018Q1
get_equity： 9945 2018 1


 98%|████████████████████████████████████████████████████████████████████████████████▍ | 52/53 [01:32<00:01,  1.93s/it]

9945 查無資料
9958 已存在


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [01:33<00:00,  1.76s/it]


In [17]:
# 計算歷年成長%
df_y_all = pd.DataFrame()

for lst_id in lst_all_id:
    for stock_id in tqdm(lst_id):
        path_equity = root + 'equity/quarterly_equity_{}.xlsx'.format(stock_id)
        if not os.path.exists(path_equity):
            print(stock_id, '不存在')
            continue
        df_quarterly = pd.read_excel(path_equity, engine='openpyxl')
        df_quarterly = df_quarterly.set_index(df_quarterly.columns[0])
        t_wait = np.random.randint(5,10)
        print(stock_id)
        df_y = check_5_years(stock_id, df_quarterly, df_today, True)
        df_y_all = df_y_all.append(df_y)
        if not df_y is None:
            df_y.to_excel(path_equity.replace('quarterly_','yearly/yearly_'))
    
    path = root+'equity/yearly/check/finance_{}.xlsx'.format(datetime.now().strftime('%Y_%m_%d_%H%M'))
    df_y_all.to_excel(path)
    print(path)
df_y_all.head()

  0%|                                                                                           | 0/55 [00:00<?, ?it/s]

1101
1101  wait  9


  2%|█▌                                                                                 | 1/55 [00:09<08:26,  9.38s/it]

現金股利_成長(%)< -20
1102
1102  wait  9


  4%|███                                                                                | 2/55 [00:18<08:17,  9.39s/it]

...

 98%|████████████████████████████████████████████████████████████████████████████████▍ | 52/53 [06:13<00:07,  7.02s/it]

EPS(Y)_成長(%) < -20
9958
9958  wait  6


100%|██████████████████████████████████████████████████████████████████████████████████| 53/53 [06:19<00:00,  7.16s/it]

稅後淨利_成長(%)< -20


,stock_id,2015,2016,2017,2018,2019,2020,2021,股價,預估EPS(Y),...,殖利率(%),盈餘成長率(%),總報酬率(%),總報酬本益比,lowest_est,highest_est,本益比_min_avg,本益比_max_avg,2022,2014
營業收入(億),1102,662.9,609.5,649.0,827.4,893.5,782.4,618.1,48.5,4.7,...,7.1,16.64,23.74,2.3,23.35,60.71,5.0,13.0,NaN,NaN
營業收入_成長(%),1102,0.0,-8.1,6.5,27.5,8.0,-12.4,-21.0,48.5,4.7,...,7.1,16.64,23.74,2.3,23.35,60.71,5.0,13.0,NaN,NaN
稅後淨利(億),1102,49.1,46.6,66.3,148.7,221.5,188.3,82.0,48.5,4.7,...,7.1,16.64,23.74,2.3,23.35,60.71,5.0,13.0,NaN,NaN
稅後淨利_成長(%),1102,0.0,-4.9,42.1,124.3,48.9,-15.0,-56.5,48.5,4.7,...,7.1,16.64,23.74,2.3,23.35,60.71,5.0,13.0,NaN,NaN
股東權益,1102,0.0,0.0,364.2,1550.0,1658.8,1654.2,1280.8,48.5,4.7,...,7.1,16.64,23.74,2.3,23.35,60.71,5.0,13.0,NaN,NaN


In [23]:
path_check = root+'equity/yearly/check/finance_20220330_2.xlsx'
df_check = pd.read_excel(path_check, engine='openpyxl')
print(len(df_check))
df_check.head(2)

651


,Unnamed: 0,stock_id,2015,2016,2017,2018,2019,2020,2021,股價,...,殖利率(%),盈餘成長率(%),總報酬率(%),總報酬本益比,lowest_est,highest_est,本益比_min_avg,本益比_max_avg,2022,2014
0,營業收入(億),1102,662.9,609.5,649.0,827.4,893.5,782.4,618.1,48.5,...,7.1,16.64,23.74,2.3,23.35,60.71,5.0,13.0,NaN,NaN
1,營業收入_成長(%),1102,0.0,-8.1,6.5,27.5,8.0,-12.4,-21.0,48.5,...,7.1,16.64,23.74,2.3,23.35,60.71,5.0,13.0,NaN,NaN


In [24]:
lst_threshold = [-10, -5]
for threshold in lst_threshold:
    lst_remove_id = []
    for stock_id in df_check['stock_id'].unique():
        df_stock = df_check[df_check['stock_id']==stock_id]
        df_stock = df_stock[(df_stock[2017]<threshold) | (df_stock[2018]<threshold) \
                               | (df_stock[2019]<threshold) | (df_stock[2020]<threshold)]
        if len(df_stock)>0:
            print(stock_id, '不符合')
            lst_remove_id.append(stock_id)
    print(lst_remove_id)
    df_check_05 = df_check[~df_check['stock_id'].isin(lst_remove_id)]
    print(len(df_check_05))
    s_now = datetime.now().strftime('%Y_%m_%d_%H%M')
    path = root+'equity/yearly/check/finance_check_{}_{}.xlsx'.format(str(threshold*-1), s_now)
    df_check_05.to_excel(path, index=False)

1102 不符合
1476 不符合
1504 不符合
1513 不符合
2330 不符合
2395 不符合
2454 不符合
2886 不符合
3005 不符合
3653 不符合
3665 不符合
5306 不符合
6202 不符合
6409 不符合
8016 不符合
9938 不符合
9939 不符合
[1102, 1476, 1504, 1513, 2330, 2395, 2454, 2886, 3005, 3653, 3665, 5306, 6202, 6409, 8016, 9938, 9939]
294
1102 不符合
1476 不符合
1504 不符合
1513 不符合
2330 不符合
2395 不符合
2428 不符合
2454 不符合
2886 不符合
3005 不符合
3653 不符合
3665 不符合
5306 不符合
6202 不符合
6239 不符合
6409 不符合
6592 不符合
8016 不符合
9938 不符合
9939 不符合
[1102, 1476, 1504, 1513, 2330, 2395, 2428, 2454, 2886, 3005, 3653, 3665, 5306, 6202, 6239, 6409, 6592, 8016, 9938, 9939]
231


In [5]:
stock_id = '3592'
path_equity = root + 'equity/quarterly_equity_{}.xlsx'.format(stock_id)
df_quarterly = pd.read_excel(path_equity, engine='openpyxl')
df_quarterly = df_quarterly.set_index(df_quarterly.columns[0])
t_wait = np.random.randint(5,10)
print(stock_id)
df_y = check_5_years(stock_id, df_quarterly, df_today, True)
df_y

3592
上櫃公司
this year 2021
3592  wait  8
No tables found


year,stock_id,2021,2022
營業收入(億),3592,67.4,0.0
營業收入_成長(%),3592,0.0,-100.0
稅後淨利(億),3592,12.6,0.0
稅後淨利_成長(%),3592,0.0,0.0
股東權益,3592,17.8,0.0
ROE(%),3592,70.6,0.0
毛利率(%),3592,46.3,0.0
毛利率_成長(%),3592,0.0,0.0
營益率(%),3592,22.0,0.0
淨利率(%),3592,22.5,0.0


In [10]:
lst_stock_id = ['2379']
df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍
df_quarterly_all = get_df_quarterly_all()
df_estimate = pd.DataFrame(columns = list(df_quarterly_all.columns)+['普通股數'])

# 取得公司發行股數
path_company_stock_num = root + 'company_list.xlsx'
df_company_stock_num = pd.read_excel(path_company_stock_num, engine='openpyxl', dtype={'stock_id':str})
df_company_stock_num.set_index('stock_id', inplace=True)

lst_eps = []
for i in tqdm(range(len(lst_stock_id))):
    stock_id = lst_stock_id[i]
    t_wait = np.random.randint(5,15)
    print(stock_id, ', wait ', t_wait)
    df = get_df_quarterly(stock_id)

    lst_eps.append(stock_id)
    # 計算歷年同季的營收成長率、EPS
    df = set_growth_eps(df, df_today, stock_id)

    # 將年度EPS寫到歷年excel裡
    lst_year = set(df.index.to_series().apply(lambda x:x[:4]))
    min_year = int(min(lst_year)) # 若最小年是2018 ，只能確定2019都有資料
    lst_year = [y for y in lst_year if int(y) > min_year]
    lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]
    df.reset_index(inplace=True)
    df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

    print(lst_EPS)
    stock_name = df_today.loc[stock_id, 'name']
    df_yearly = get_yearly(stock_id, lst_EPS, df['quarter'][0][:4], stock_name)    

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

2379 , wait  9


c:\users\mia\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[{'year': '2019', 'eps': 13.36}, {'year': '2018', 'eps': 8.57}, {'year': '2016', 'eps': 6.02}, {'year': '2015', 'eps': 4.81}, {'year': '2021', 'eps': 31.97}, {'year': '2020', 'eps': 17.24}, {'year': '2017', 'eps': 6.71}]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.99s/it]


In [9]:
df_yearly

,stock_id,name,year,highest,lowest,average,EPS,本益比_max,本益比_min
0,2379,瑞昱,2021,799.25,319.70,508.23,31.97,25.0,10.0
1,2379,瑞昱,2020,432.00,162.00,312.62,17.24,25.1,9.4
2,2379,瑞昱,2019,251.50,136.00,207.43,13.36,18.8,10.2
3,2379,瑞昱,2018,151.00,101.00,123.94,8.57,17.6,11.8
4,2379,瑞昱,2017,124.50,99.50,109.36,6.71,18.6,14.8
5,2379,瑞昱,2016,133.00,71.30,98.43,6.02,22.1,11.8
6,2379,瑞昱,2015,108.00,49.75,80.85,4.81,22.5,10.3


In [16]:
df_yearly[df_yearly['year']=='2021']

,stock_id,name,year,highest,lowest,average,EPS,本益比_max,本益比_min
0,2379,瑞昱,2021,799.25,319.7,508.23,31.97,25.0,10.0


In [13]:
stock_id = '2379'
path_equity = root + 'equity/quarterly_equity_{}.xlsx'.format(stock_id)
df_quarterly = pd.read_excel(path_equity, engine='openpyxl')
df_quarterly = df_quarterly.set_index(df_quarterly.columns[0])
df_quarterly

,year,營業收入,營業成本,營業毛利,營業毛利(毛損)淨額,營業費用,營業利益,稅前淨利,所得稅費用,歸屬母公司淨利(損),每股盈餘(元),股東權益
Unnamed: 0,,,,,,,,,,,,
2021Q3,2021,289.55,137.16,152.39,152.39,98.86,53.53,50.92,2.04,48.88,9.57,345
2021Q2,2021,258.39,128.14,130.25,130.25,87.24,43.01,44.85,1.80,43.05,8.43,294
2021Q1,2021,233.41,128.88,104.53,104.53,74.60,29.93,31.83,1.28,30.55,5.98,331
2020Q4,2020,220.84,125.86,94.99,94.99,68.47,26.52,27.91,1.66,26.25,5.14,294
2020Q3,2020,224.09,131.43,92.66,92.66,66.96,25.70,26.70,1.61,25.09,4.92,274
2020Q2,2020,173.38,95.79,77.59,77.59,57.91,19.68,21.59,1.32,20.28,3.97,251
2020Q1,2020,159.28,92.04,67.24,67.24,52.75,14.50,17.36,1.05,16.31,3.21,288
2019Q4,2019,166.85,93.58,73.27,73.27,55.37,17.90,17.59,1.15,16.43,3.23,272
2019Q3,2019,160.43,91.64,68.80,68.80,52.07,16.72,20.47,1.24,19.22,3.78,266


In [14]:
lst_quarter = [str(this_year) + 'Q' + str(i) for i in range(1,5)]
num_quarter = len(df_quarterly[df_quarterly.index.isin(lst_quarter)]) # 已公布的季報數量

lst_year = [2018, 2019, 2020, 2021]
lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]

df.reset_index(inplace=True)
df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

df_yearly = get_yearly(stock_id, lst_EPS, '2021', "")  

['2021Q1', '2021Q2', '2021Q3', '2021Q4'] 3


31.97

In [5]:
# 計算歷年成長%
stock_id = '2379'
path_equity = root + 'equity/quarterly_equity_{}.xlsx'.format(stock_id)

df_quarterly = pd.read_excel(path_equity, engine='openpyxl')
df_quarterly = df_quarterly.set_index(df_quarterly.columns[0])
df_quarterly.head(1)

,year,營業收入,營業成本,營業毛利,營業毛利(毛損)淨額,營業費用,營業利益,稅前淨利,所得稅費用,歸屬母公司淨利(損),每股盈餘(元),股東權益
Unnamed: 0,,,,,,,,,,,,
2021Q3,2021,289.55,137.16,152.39,152.39,98.86,53.53,50.92,2.04,48.88,9.57,345
